## VA Diabetes 

In [1]:
# Package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import missingno as msno
import re
import os

pd.set_option('display.max_columns', None)
color_pal = sns.color_palette()
%matplotlib inline

In [2]:
file_path = '/Users/ben/Projects/VA_diab_review/Model Database for Share (2).xlsx'

In [3]:
# Load the Excel file into a pandas DataFrame
df = (
    pd.read_excel(file_path)
    # Dropping columns with 0 non-null count
    .dropna(axis=1, how='all')
    # Renaming 'Unnamed: 0' to 'Subject'
    .rename(columns={'Unnamed: 0': 'Subject'})
    # Renaming 'medications_(gluc,_bp,_lipid,_statin)' to 'medications'
    .rename(columns={'medications_(gluc,_bp,_lipid,_statin)': 'medications'})
    # Lowercasing and replacing spaces with underscores in all column names
    # Also removing any leading spaces
    .rename(columns=lambda x: x.strip().lower().replace(' ', '_'))
)

# Display basic information about the DataFrame
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278 entries, 0 to 1277
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   subject                                471 non-null    float64       
 1   date_of_visit                          688 non-null    datetime64[ns]
 2   age                                    1125 non-null   float64       
 3   sex                                    469 non-null    object        
 4   wt                                     688 non-null    float64       
 5   sbp                                    681 non-null    float64       
 6   dbp                                    681 non-null    object        
 7   bmi                                    644 non-null    object        
 8   obese                                  471 non-null    object        
 9   pre-dm                                 464 non-null    object  

,subject,date_of_visit,age,sex,wt,sbp,dbp,bmi,obese,pre-dm,dm_t2,a1c,trig,hdl,total_chol,ldl,"medications_(gluc,_bp,_lipid,_statin)",cgm,able_to_adhere
0,1.0,2023-03-27,123.0,Male,264.0,129.0,78,39.07,YES,NO,YES,6.4,187.0,54,113.0,22.1,"Atorvastatin, Losartan, Metformin,",NaN,NaN
1,NaN,2023-06-08,123.0,NaN,258.0,116.0,62,NaN,NaN,NaN,NaN,6.3,NaN,NaN,NaN,NaN,No med changes,NaN,NaN
2,NaN,2023-10-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,NaN,NaN,NaN,No med changes,NaN,NaN
3,NaN,2023-11-14,123.0,NaN,265.0,130.0,78,39.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,2023-03-28,123.0,Male,233.0,130.0,89,31.67,YES,NO,YES,6.7,63.0,49,146.0,84.6,"Lisinopril, alogliptin, empagliflozin, hctz/tr...",NaN,NaN


### Cleanup 

In [4]:
# Drop rows where 'wt' column has NaN values
df= df.dropna(subset=['wt','date_of_visit'])

In [5]:
df.isna().sum()

subject                                  212
date_of_visit                              0
age                                       34
sex                                      224
wt                                         0
sbp                                        8
dbp                                        8
bmi                                       42
obese                                    211
pre-dm                                   218
dm_t2                                    218
a1c                                      158
trig                                     199
hdl                                      199
total_chol                               199
ldl                                      201
medications_(gluc,_bp,_lipid,_statin)    300
cgm                                      673
able_to_adhere                           681
dtype: int64

In [6]:
# drop columns with majority of values missing / redundant
df = df.drop(columns=['age', 'sex', 'cgm', 'able_to_adhere'])

In [7]:
# Forward filling NaN values in the 'subject' column and sorting by 'subject' and 'date_of_visit'
df['subject'] = df['subject'].ffill()
df = df.sort_values(by=['subject', 'date_of_visit'])

# Creating the 'appointment_number' column with the updated 'subject' column
df['appointment_number'] = df.groupby('subject').cumcount() + 1

df.head()


,subject,date_of_visit,wt,sbp,dbp,bmi,obese,pre-dm,dm_t2,a1c,trig,hdl,total_chol,ldl,"medications_(gluc,_bp,_lipid,_statin)",appointment_number
0,1.0,2023-03-27,264.0,129.0,78,39.07,YES,NO,YES,6.4,187.0,54,113.0,22.1,"Atorvastatin, Losartan, Metformin,",1
1,1.0,2023-06-08,258.0,116.0,62,NaN,NaN,NaN,NaN,6.3,NaN,NaN,NaN,NaN,No med changes,2
3,1.0,2023-11-14,265.0,130.0,78,39.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2.0,2023-03-28,233.0,130.0,89,31.67,YES,NO,YES,6.7,63.0,49,146.0,84.6,"Lisinopril, alogliptin, empagliflozin, hctz/tr...",1
5,2.0,2023-06-27,210.0,139.0,83,NaN,NaN,NaN,NaN,5.3,NaN,NaN,NaN,NaN,"Lisinopril, amlodipine,",2


In [8]:
df.to_csv('df.csv', index=False)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 682 entries, 0 to 1164
Data columns (total 16 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   subject                                682 non-null    float64       
 1   date_of_visit                          682 non-null    datetime64[ns]
 2   wt                                     682 non-null    float64       
 3   sbp                                    674 non-null    float64       
 4   dbp                                    674 non-null    object        
 5   bmi                                    640 non-null    object        
 6   obese                                  471 non-null    object        
 7   pre-dm                                 464 non-null    object        
 8   dm_t2                                  464 non-null    object        
 9   a1c                                    524 non-null    object        

In [10]:
def clean_a1c(value):
    if pd.isna(value):
        return value
    value = str(value)  # Convert to string to handle NaN values
    value = re.sub(r"\(.*\)", "", value)  # Remove anything in parentheses
    try:
        return float(value)
    except ValueError:
        return None

# Apply the function to the 'a1c' column and assign it back to the same column in df
df['a1c'] = df['a1c'].apply(clean_a1c)


In [11]:
df.a1c.unique()

array([ 6.4 ,  6.3 ,   nan,  6.7 ,  5.3 ,  9.  ,  6.5 ,  5.7 ,  6.9 ,
        5.5 ,  5.  ,  6.1 ,  5.8 ,  5.6 ,  6.  ,  5.4 ,  7.2 ,  5.1 ,
        8.5 ,  6.8 ,  5.9 ,  5.2 ,  9.2 ,  6.6 ,  4.7 ,  8.6 ,  7.7 ,
        4.4 ,  9.4 ,  4.9 ,  7.3 ,  7.5 ,  4.8 ,  8.8 ,  6.2 ,  9.5 ,
        7.1 , 10.6 ,  4.6 ,  7.9 ,  8.3 ,  9.1 ,  8.1 ,  7.8 ,  8.  ,
        7.6 ,  7.  ,  9.7 ,  9.8 , 10.7 , 10.4 ,  0.52,  4.5 , 10.9 ])

In [12]:
df.head()

,subject,date_of_visit,wt,sbp,dbp,bmi,obese,pre-dm,dm_t2,a1c,trig,hdl,total_chol,ldl,"medications_(gluc,_bp,_lipid,_statin)",appointment_number
0,1.0,2023-03-27,264.0,129.0,78,39.07,YES,NO,YES,6.4,187.0,54,113.0,22.1,"Atorvastatin, Losartan, Metformin,",1
1,1.0,2023-06-08,258.0,116.0,62,NaN,NaN,NaN,NaN,6.3,NaN,NaN,NaN,NaN,No med changes,2
3,1.0,2023-11-14,265.0,130.0,78,39.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2.0,2023-03-28,233.0,130.0,89,31.67,YES,NO,YES,6.7,63.0,49,146.0,84.6,"Lisinopril, alogliptin, empagliflozin, hctz/tr...",1
5,2.0,2023-06-27,210.0,139.0,83,NaN,NaN,NaN,NaN,5.3,NaN,NaN,NaN,NaN,"Lisinopril, amlodipine,",2


In [13]:
# Add 'date_of_visit' to the aggregation
grouped_df = df.groupby('subject').agg({'wt': ['first', 'last'], 
                                        'a1c': ['first', 'last'], 
                                        'sbp': ['first', 'last'], 
                                        'dbp': ['first', 'last'], 
                                        'date_of_visit': ['first', 'last'],
                                        'appointment_number': 'count'}).reset_index()

# Flattening the MultiIndex for columns
grouped_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in grouped_df.columns.values]

# Renaming columns for clarity
grouped_df.rename(columns={
    'wt_first': 'pre_intervention_wt',
    'wt_last': 'post_intervention_wt',
    'a1c_first': 'pre_intervention_a1c',
    'a1c_last': 'post_intervention_a1c',
    'sbp_first': 'pre_intervention_sbp',
    'sbp_last': 'post_intervention_sbp',
    'dbp_first': 'pre_intervention_dbp',
    'dbp_last': 'post_intervention_dbp',
    'date_of_visit_first': 'first_visit_date',
    'date_of_visit_last': 'last_visit_date'
}, inplace=True)

# Creating the 'tbi' column representing the difference between the last and first date_of_visit
grouped_df['tbi'] = pd.to_datetime(grouped_df['last_visit_date']) - pd.to_datetime(grouped_df['first_visit_date'])



In [14]:
grouped_df['subject'] = grouped_df['subject'].astype('object')


In [15]:
grouped_df.to_csv('df.csv', index=False)

In [16]:
grouped_df.columns

Index(['subject', 'pre_intervention_wt', 'post_intervention_wt',
       'pre_intervention_a1c', 'post_intervention_a1c', 'pre_intervention_sbp',
       'post_intervention_sbp', 'pre_intervention_dbp',
       'post_intervention_dbp', 'first_visit_date', 'last_visit_date',
       'appointment_number_count', 'tbi'],
      dtype='object')

In [17]:
df.columns

Index(['subject', 'date_of_visit', 'wt', 'sbp', 'dbp', 'bmi', 'obese',
       'pre-dm', 'dm_t2', 'a1c', 'trig', 'hdl', 'total_chol', 'ldl',
       'medications_(gluc,_bp,_lipid,_statin)', 'appointment_number'],
      dtype='object')

### EDA

### Hypothesis

In [18]:

from scipy.stats import ttest_rel

# Count the number of visits for each ID
visit_counts = bp.groupby('subject').size()

# Filter out IDs with only one visit
data_multiple_visits = bpa[bpa['subject'].isin(visit_counts[visit_counts > 1].index)]

# Group by ID and calculate the weight change for patients with multiple visits
weight_change = data_multiple_visits.groupby('subject')['wt'].agg(['first', 'last'])
weight_change['weight_change'] = weight_change['last'] - weight_change['first']

# Filter out rows where either 'first' or 'last' weight is NaN
weight_change_filtered = weight_change.dropna(subset=['first', 'last'])

# Perform the paired t-test only if there are enough pairs
if len(weight_change_filtered) > 1:
    t_stat, p_value = ttest_rel(weight_change_filtered['first'], weight_change_filtered['last'])
    print("t-statistic:", t_stat)
    print("p-value:", p_value)

NameError: name 'bp' is not defined